# Face Detection
## Unstructured Data Image proyect
### By
- Juan Miguel Ramos Pugnaire
- Juan Ortega Ortega
- Pablo Cardenal

### Objective

The aim of this project is to create a mock face detection software, in which an interface will be used to interact with the convolutional neural network we will train. It will be able to differentiate between five different classes:

- The three faces of all group members (quite a challegene, as we all have a beard and black hair!)
- A random face that does not belong to our group.
- An image without a face in it.

Once the objective is clear we can move on to the description of the project and the challenges we have faced.

### Solutions

The report and general workflow of the project has been structured in the following way:

__Project Index__
<ol>
<li>Data Extraction</li>
<li>Interface development</li>
<li>Training & Test</li>
<li>Parameter Tuning</li>
<li>Results</li>
</ol>

In [ ]:
import json
training_files="./training"
test_files="./test"

### 1. Data Extraction 

Once the idea for the project was clear the first thing we had to do was getting a good quality dataset. Thankfully there is a plethora of datasets made for fface detection and we decided to use the one made by Gwangbin Bae, Martin de La Gorce, et al. on their paper

> [DigiFace-1M: 1 Million Digital Face Images for Face Recognition](https://github.com/microsoft/DigiFace1M/raw/main/paper.pdf)

the dataset is available for download on [their github page](https://github.com/microsoft/DigiFace1M). As the title of the paper may suggest, this is an artificially generated dataset of faces in which they aim to reduce the problems of dataset crawled from the web. They sey those datasets "are severely biased (in terms of race, lighting, make-up, etc) and often contain label noise", as well as posing ethical problems. Thus, we decided to go with a reduced version of this dataset. A sample of it can be seen on the following figure:

![Screenshot of the sample dataset](FullFaceDataSample.png)

Figure 1: Screenshot of the sample dataset.

We wanted the model to train with the highest amount of different faces so we took a fragment of the whole dataset, consisting of 5 different expressions per face. This accounted for about 206.000 different pictures, far more than what we will need. 

This, however, poses another problem as we need to take our pictures in the same format as the chosen dataset; that is, we need to crop a picture of our faces, roughfly centered on a 112x112 pixel square. To overcome this problem we created a simple script, `dataCapture/PictueExtracion.py`, that takes care of it. using the `open-cv` python library we use our laptop webcam and save a 224x224 square that will later be resized. To help centering our faces we overlaid a blue square in the area that will later be saved. Originally, we took a 112x112 region but it was difficult and uncomfortable to take pictures. Thus, we decided to take a bigger area and resize it to fit the pixel dimensions of the original data.

### 2. Data Preparation 
*Altering the training and test*  
Parameters in data preparation:
- directory : Location of the Directory
- labels="inferred": labels of the photos stated by the directory structure
- label_mode= "categorical" : Type of label
- class_names= ["JuanM","JuanO","Pablo","pepe"]: Only valid if label is inferred
- color_mode= "grayscale", "rgb", "rgba" depending on channel
- batch_size=32 : Number of images tat are proccessed in each iteration
- image_size=(256,256)(default): Size to resize the image when rea from disk-> Un batchsize pequeño es más preciso pero más lento 
- shuffle=True(default)
- seed=optional 
- validation_split= 0.1 :Fraction of data reserved for validation
- subset: only used when valdation is set
- interpolation=bilinear(default) : used for resizing images 
- crop_to_aspect_ratio: True para redimensión recortando imagen, False para redimensión con deformidad



In [ ]:
def save(model, filename: str, model_name: str):
    """Saves the model to an .h5 file and the model name to a .json file.

    Args:
        filename: Relative path to the file without the extension.

    """
    # Save Keras model
    model.save(filename + '.h5')

    # Save base model information
    with open(filename + '.json', 'w', encoding='utf-8') as f:
        json.dump(model_name, f, ensure_ascii=False, indent=4, sort_keys=True)

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np

print(tf.__version__)

#--------------------- Data Preprocessing --------------------#

#feature training
train_datagen = ImageDataGenerator(
        # reducing/normalizing the pixels
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
#connecting the image augmentation tool to our dataset
train_set = train_datagen.flow_from_directory(
        './training',
        #final size of the images that will be fed into the ann
        target_size=(640, 480),
        # number of images that we want to have in each batch
        batch_size=32,
        # we have binary classification --> binary class mode
        class_mode='categorical')


#only rescaling but no transformations
test_datagen = ImageDataGenerator(rescale=1./255)
#connecting to the test data
test_set = test_datagen.flow_from_directory(
        './test',
        target_size=(640, 480),
        batch_size=32,
        class_mode='categorical')

print(test_set)

### 3. Interface develpment
*Interface programming*  
[Enter explanation]


### 4. Training & Test
*First model structure*  
[Enter explanation]


In [ ]:
#--------------------- Building CNN --------------------#
# initializing CNN as sequential layers
from tf.keras.callbacks import EarlyStopping
def create_model():
        
    cnn = tf.keras.models.Sequential()

    # Step 1: Convolution to get the Feature Map
    cnn.add(tf.keras.layers.Conv2D(filters= 210 , kernel_size = 2, activation = 'relu', input_shape=[640,480,3]))
    # filters: output feature map
    # kernel_size: size of the feature detector
    # strides: step size from one filter to the next default is 1
    # Step 2: Max Pooling
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2 ,strides=2))
    #adding a second convolutional layer
    cnn.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu'))
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2 ,strides=2))
    #adding a second convolutional layer
    cnn.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu'))
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2 ,strides=2))
    #adding a second convolutional layer
    cnn.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu'))
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2 ,strides=2))
    # Step 3: Flattening
    cnn.add(tf.keras.layers.Flatten())

    # Step 4: Full Connection
    cnn.add(tf.keras.layers.Dense(units = 256, activation = 'relu'))

    # Step 5: Output Layer
    cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))
    return cnn

#--------------------- Training the CNN --------------------#
#compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#training the CNN on the training set and evaluating it on the test set
cnn.fit(x = train_set, validation_data = test_set, epochs = 25)

early_stopping_monitor = EarlyStopping(patience=10)
history = model.fit(X, y, validation_split=0.33, epochs=200, batch_size=15, verbose=0, callbacks=[early_stopping_monitor])


### 5. Parameter tuning
*Detail parameters and how they change*  
[Enter explanation]


### 6. Final Results
*Check final results and future implementations*  
[Enter explanation]
